In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import spacy
import json
import gensim
from climdist.data import load as cd_load
import string

In [2]:
df = cd_load('main', readability=True, heading2=True)

## improve heading2

In [3]:
#from climdist.data_preprocess import generate_alt_headings

In [4]:
nlp = spacy.load('../data/models/spacy_ner_151121/model-best/')

In [60]:
def alt_heading(heading):
    doc = nlp(heading)
    alt_heading = ''
    locs = [ent for ent in doc.ents if ent.label_ == 'LOC']
    if len(locs) > 0:
        print(locs)
        if len(locs) == 1 or locs[0].start == 0:
            alt_heading = locs[0].text
                
    return alt_heading

In [46]:
def generate_alt_headings(df, ignorelocs, nlp,
                          output_path=None):

    """Input: main df with normalized headings. Use a trained NLP model for detecting LOC entities in headings.
        Output: column with alternate headings for simpler geographical analysis.
        Can take 1-2 hours."""
    
    df['heading2'] = df['heading']
    
    for i, heading in tqdm(df.heading2.iteritems()):
        doc = nlp(heading)
        locs = [ent for ent in doc.ents if ent.label_ == 'LOC']
        if len(locs) > 0:
            if len(locs) == 1 or locs[0].start == 0:
                if locs[0].text not in ignorelocs:
                    df.loc[i, 'heading2'] = locs[0].text

    if output_path:
        df['heading2'].to_parquet(output_path, index=True)
    
    return df['heading2']

In [19]:
with open('../temp/typhon_index.json', 'r', encoding='utf8') as f:
    typhon_index = json.load(f)

In [48]:
typhon_headings = generate_alt_headings(typhon_df, ignorelocs= ['Riga'], nlp=nlp)

234it [00:01, 154.46it/s]


In [52]:
example_heading = df.heading2[df.heading2.str.contains('Japan. Ueber San FranziSto und Newyork treffen N', na=False)]

In [62]:
alt_heading('Japan. Yokuhama, 2. Dec. (20. November). Der M...')

[Yokuhama]


'Yokuhama'

In [31]:
typhon_df = df.loc[typhon_index]

In [37]:
typhon_df

,date,year,month,day,pub,heading,full_text,href,text_len,readability,heading2
147243,1860-04-20,1860,4,20,Rigasche Zeitung,Inländische Nachrichten. Inländische Nachrichte«.,Inländische Nachrichten. Inländische Nachricht...,https://proc.dom.lndb.lv/file.axd?id=11404404&...,7484,1,Inländische Nachrichten. Inländische Nachrichte«.
147243,1860-04-20,1860,4,20,Rigasche Zeitung,Inländische Nachrichten. Inländische Nachrichte«.,Inländische Nachrichten. Inländische Nachricht...,https://proc.dom.lndb.lv/file.axd?id=11404404&...,7484,1,Inländische Nachrichten. Inländische Nachrichte«.
147243,1860-04-20,1860,4,20,Rigasche Zeitung,Inländische Nachrichten. Inländische Nachrichte«.,Inländische Nachrichten. Inländische Nachricht...,https://proc.dom.lndb.lv/file.axd?id=11404404&...,7484,1,Inländische Nachrichten. Inländische Nachrichte«.
149003,1860-09-28,1860,9,28,Rigasche Zeitung,Das Journal des russischen Marine – Ministeriu...,Das Journal des russischen Marine – Ministeriu...,https://proc.dom.lndb.lv/file.axd?id=10724868&...,4551,1,Das Journal des russischen Marine – Ministeriu...
155896,1862-04-30,1862,4,30,Rigasche Zeitung,"* London, 4. Mai (22. April). Die „Morning-Pos...","* London, 4. Mai (22. April). Die „Morning-Pos...",https://proc.dom.lndb.lv/file.axd?id=7690943&a...,3601,1,"* London, 4. Mai (22. April). Die „Morning-Pos..."
...,...,...,...,...,...,...,...,...,...,...,...
192542,1869-08-02,1869,8,2,Rigasche Zeitung,Asien.,"Asien.\n\n\tJapan. Die ~A. A. Z."" Heilt mehrer...",https://proc.dom.lndb.lv/file.axd?id=10865501&...,12634,1,Asien.
193347,1869-09-19,1869,9,19,Rigasche Zeitung,Yokuhama 26. (14 ) Juli. Am 3. d. M. ist das O...,Yokuhama 26. (14 ) Juli. Am 3. d. M. ist das O...,https://proc.dom.lndb.lv/file.axd?id=2573703&a...,6188,1,Yokuhama 26. (14 ) Juli. Am 3. d. M. ist das O...
193639,1869-10-07,1869,10,7,Rigasche Zeitung,Asien.,Asien.\n\n\tYoknham ~ 10. Aug. (29. Juli.) Es ...,https://proc.dom.lndb.lv/file.axd?id=4711227&a...,2852,1,Asien.
194971,1869-12-22,1869,12,22,Rigasche Zeitung,"Japan. Yokuhama, 2. Dec. (20. November). Der M...","Japan. Yokuhama, 2. Dec. (20. November). Der M...",https://proc.dom.lndb.lv/file.axd?id=10363325&...,586,1,"Japan. Yokuhama, 2. Dec. (20. November). Der M..."


## Vector average

In [65]:
def vector_mean(keys):
    
    sum_vec = np.zeros(100,)
    for key in keys:
        if model.has_index_for(key):
            sum_vec += model.get_vector(key)
        
    mean_vec = sum_vec/len(keys)
        
    #print(model.most_similar(mean_vec))
    return mean_vec

In [142]:
def create_heading_vectors(df):
    
    vectors = []
    
    for heading in tqdm(df.heading):
        heading_vec = list(vector_mean(heading.strip('.').split()))
        heading_vec.insert(0, heading.strip('.'))
        vectors.append(heading_vec)
        
    return pd.DataFrame(vectors, columns=(['heading']+list(range(100)))).set_index('heading')

In [204]:
def create_heading_vectors_str(df):
    
    vectors = ''
    vectors += (str(len(df.heading2.unique()))) + ' 100\n' #write meteadata
    
    for heading in tqdm(df.heading2.unique()):
        try:
            heading_vec = vector_mean(heading.strip('.').split())
            line = heading.replace(' ', '_') + ' ' + ' '.join([str(num) for num in heading_vec])
            vectors += line
            vectors += '\n'
        except:
            pass
        
    return vectors

In [8]:
def format_heading(heading):
    return heading.strip('.').replace(' ', '_')

In [205]:
vectors = create_heading_vectors_str(df)

 61%|█████████████████████████████████████████████▏                            | 52951/86598 [51:11<1:07:15,  8.34it/s]C:\Users\krister\AppData\Local\Temp/ipykernel_27420/3645892505.py:8: RuntimeWarning: invalid value encountered in true_divide
  mean_vec = sum_vec/len(keys)
100%|██████████████████████████████████████████████████████████████████████████| 86598/86598 [2:11:33<00:00, 10.97it/s]


In [ ]:
with open('../temp/heading_vectors.txt', 'w', encoding='utf8') as f:
    f.write(vectors)

In [2]:
heading_model = gensim.models.word2vec.KeyedVectors.load_word2vec_format('../temp/heading_vectors.txt')

In [9]:
for heading in df.heading2.sample(10):
    print(heading) 
    print(heading_model.most_similar(format_heading(heading)))
    print('\n\n')

Oesterreich
[('Oesterreich_–_Ungarn', 0.946531355381012), ('Oesterreich_Ungarn', 0.946531355381012), ('Oesterreich-Ungarn', 0.8309103846549988), ('Preußen', 0.811760663986206), ('Ungarn', 0.7910464406013489), ('Socialpädagogisches_aus_Deutschland', 0.7641233205795288), ('Deutschland', 0.7641233205795288), ('—_Zwischen_den_Höfen_von_Dänemark_und_Preußen_ist', 0.7450395226478577), ('Italien', 0.737226128578186), ('Frankreich', 0.6938095688819885)]



Waaren-Preise in Silber-Rubeln


KeyError: "Key 'Waaren-Preise_in_Silber-Rubeln' not present"

## doc2vec

In [16]:
nlp = spacy.load('de_core_news_md')

In [21]:
def token_is_garbage(token, treshold=0.3):
    
    allowed_symbols = string.ascii_letters + '0123456789.äüö'
    
    non_alphabetical = 0
    for symbol in token.text:
        if symbol not in allowed_symbols:
            non_alphabetical += 1
            
    if non_alphabetical/len(token.text) >= treshold:
        return True
    else:
        return False
    

def cleanup_heading(heading, nlp):
    
    doc = nlp(heading)
    word_list = []
    for token in doc:
        if not token.is_stop and token.pos_ not in ['PUNCT', 'SPACE', 'NUM']:
            if token_is_garbage(token) == False:
                word_list.append(token.text)

    return word_list

In [22]:
df.heading2.sample(100).apply(lambda x: cleanup_heading(x, nlp))

78984                  [bcrclls, besprochene, dramamme]
98604                                     [Deutschland]
73697                                 [St., Petersburg]
18525                                        [Verlören]
86660                                      [Frankreich]
                              ...                      
49898                                 [St., Petersburg]
224388        [Riga, 2., August, fahren, iv, Reproduct]
70958                               [Bayerisches, Bier]
260010    [Riga, 27., Juli, Ueber, Reise, Kaiserlichen]
82646       [u, drucken, erlaubt, Namen, General-Gouve]
Name: heading2, Length: 100, dtype: object

In [39]:
def create_corpus(df):
    for i, heading in tqdm(df.heading2.iteritems()):
        if heading != None:
            yield gensim.models.doc2vec.TaggedDocument(cleanup_heading(heading, nlp), [i])

In [45]:
train_corpus = list(create_corpus(df.sample(50000)))

50000it [08:44, 95.34it/s] 


In [46]:
d2v = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=30)

In [47]:
d2v.build_vocab(train_corpus)

In [48]:
d2v.train(train_corpus, total_examples=d2v.corpus_count, epochs=d2v.epochs)

In [59]:
def similarity_examples(n):
    
    for i in np.random.randint(0, len(train_corpus), n):
        print(df.loc[i, 'heading2'], '\n')
        for sim in d2v.dv.most_similar(train_corpus[100][1]):
            print(df.loc[sim[0], 'heading2'])
        print('\n\n')

In [62]:
similarity_examples(10)

Paris 

Berlin. 17. (5 ) März. In der alten Hanse« und Haf
PariS 5. Juni (24. Mai). Der Baron Lareinty. Deleg
Riga. Im Monat März dieses Jahres ist zu Lande und
Wien, 7. Januar. Die „Preise" enlbäll ans Paris di
Madrid, 27. October. AuS Lissabon wird mitgetheilt
Riga, 19. Juli. In der letzten Börsen- und Hantels
Riga 16. October. Vorgestern Mittag um 2 Uhr fand
Riga, 22. December. St. Petersburg, 21. December
P. G Konstantinopel, 4. Januar (23. December). Bek
PairSkammer. 3. April. Verhandlungen über einen Ge



Bekanntmachungen 

Berlin. 17. (5 ) März. In der alten Hanse« und Haf
PariS 5. Juni (24. Mai). Der Baron Lareinty. Deleg
Riga. Im Monat März dieses Jahres ist zu Lande und
Wien, 7. Januar. Die „Preise" enlbäll ans Paris di
Madrid, 27. October. AuS Lissabon wird mitgetheilt
Riga, 19. Juli. In der letzten Börsen- und Hantels
Riga 16. October. Vorgestern Mittag um 2 Uhr fand
Riga, 22. December. St. Petersburg, 21. December
P. G Konstantinopel, 4. Januar (23. December). Bek
PairSka